In [6]:
import os
import sys

if os.path.basename(os.getcwd()) == "testing":
    os.chdir(os.path.dirname(os.getcwd()))
    sys.path.append(os.getcwd())

import lossless.component.core.arm as arm_core
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

os.environ["OMP_NUM_THREADS"]="1"
os.environ["MKL_NUM_THREADS"]="1"
torch.set_num_threads(1)
torch.use_deterministic_algorithms(True)

DEVICE = torch.device("cpu")
arm_model = arm_core.Arm(dim_arm=16, n_hidden_layers_arm=2).to(DEVICE)
# load pretrained weights
arm_model.load_state_dict(torch.load("../logs/full_runs/trained_models/coolchic_arm.pth", map_location=DEVICE))


<All keys matched successfully>

In [3]:
import os
import sys

if os.path.basename(os.getcwd()) == "testing":
    os.chdir(os.path.dirname(os.getcwd()))
    sys.path.append(os.getcwd())


from lossless.training.manager import ImageEncoderManager
from lossless.util.config import args
from lossless.component.coolchic import CoolChicEncoder, CoolChicEncoderParameter
from lossless.util.parsecli import (
    change_n_out_synth,
    get_coolchic_param_from_args,
    get_manager_from_args,
)
from lossless.util.image_loading import load_image_as_tensor
import torch

color_space = "YCoCg"
use_image_arm = True
im_path = "../datasets/kodak/kodim17.png"
im_tensor, c_bitdepths = load_image_as_tensor(
    im_path, device="cuda:0", color_space=color_space
)

image_encoder_manager = ImageEncoderManager(**get_manager_from_args(args))
encoder_param = CoolChicEncoderParameter(
    **get_coolchic_param_from_args(args, "lossless")
)
encoder_param.set_image_size((im_tensor.shape[2], im_tensor.shape[3]))
encoder_param.layers_synthesis = change_n_out_synth(
    encoder_param.layers_synthesis, args["output_dim_size"]
)
encoder_param.use_image_arm = use_image_arm
coolchic = CoolChicEncoder(param=encoder_param)
coolchic.to_device("cuda:0")
coolchic.load_state_dict(torch.load(args["pretrained_model_path"]))

Converting image to YCoCg color space


<All keys matched successfully>

In [5]:
torch.save(coolchic.arm.state_dict(), "coolchic_arm.pth")